<a href="https://colab.research.google.com/github/kmouleejunhyuk/bigdata_analysist_practical/blob/main/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np


In [25]:
#iris data 준비
from sklearn.datasets import load_iris
dataset = load_iris()
data = dataset.data
data = pd.DataFrame(data, columns = ['a1', 'a2', 'a3', 'a4'])

In [26]:
target = dataset.target

In [27]:
#모델 준비
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [28]:
#파라미터 확인

In [29]:
# KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
#                      metric_params=None, n_jobs=None, n_neighbors=5, p=2,
#                      weights='uniform')

# RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='gini', max_depth=None, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=100,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

# XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0,
#               learning_rate=0.1, max_delta_step=0, max_depth=3,
#               min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)

In [45]:
#파라미터 그리드 준비
param_grid_rf = {
    'model__n_estimators': [100, 150],
    'model__max_depth': [None, 3, 5],
    'model__min_samples_split': [0.01, 0.05, 0.1],
    'model__max_features': ['auto', 'sqrt'],
}

param_grid_xgb = {
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__n_estimators': [100, 150],
    'model__max_depth': [3, 5]
}

param_grid_KNN = {
    'model__leaf_size': [10, 30, 50],
    'model__n_neighbors': [5, 10, 15, 20]
}

In [47]:
#파이프라인 준비 및 점수 측정
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

pipe = Pipeline([('scaler' , StandardScaler()), ('model', XGBClassifier())])
grid = GridSearchCV(pipe, param_grid_xgb, cv = 5)

score = cross_val_score(grid, data, target, scoring = 'roc_auc_ovo_weighted')
print(score)
print(score.mean())

[0.98166667 0.98333333 0.95166667 0.99166667 1.        ]
0.9816666666666667


In [ ]:
#knn: 0.993
#rf: 0.995
#XGB: 0.981
#best model: rf

In [49]:
pipe = Pipeline([('scaler' , StandardScaler()), ('model', RandomForestClassifier())])
grid = GridSearchCV(pipe, param_grid_rf, cv = 5)

#최종 결과(시험 시 제출시에만)
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.2)
grid.fit(x_train, y_train)

print(grid.best_score_)
print(grid.score(x_test, y_test))
print(grid.best_params_)

0.95
1.0
{'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_split': 0.05, 'model__n_estimators': 100}


In [50]:
grid.predict(x_test)

array([2, 1, 0, 1, 0, 2, 1, 2, 0, 1, 1, 2, 0, 1, 0, 0, 2, 0, 1, 1, 2, 0,
       0, 2, 2, 2, 2, 1, 0, 1])